In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn import metrics


In [2]:
job_category = pd.read_csv('data/job_categoy.csv',header=0)
exp = pd.read_csv('data/movie_count_by_year.csv')

In [3]:
new_head = job_category.iloc[0]
job_category = job_category[2:]
job_category.columns = new_head
job_category.head()

,category,actor,actress,archive_footage,archive_sound,cinematographer,composer,director,editor,producer,production_designer,self,writer
2,tt0000001,NaN,NaN,NaN,NaN,William Heise,NaN,William K.L. Dickson,NaN,NaN,NaN,Carmencita,NaN
3,tt0000002,NaN,NaN,NaN,NaN,NaN,Gaston Paulin,Émile Reynaud,NaN,NaN,NaN,NaN,NaN
4,tt0000003,NaN,NaN,NaN,NaN,NaN,Gaston Paulin,Émile Reynaud,Tamara Pappé,Julien Pappé,NaN,NaN,NaN
5,tt0000004,NaN,NaN,NaN,NaN,NaN,Gaston Paulin,Émile Reynaud,NaN,NaN,NaN,NaN,NaN
6,tt0000005,John Ott,NaN,NaN,NaN,NaN,NaN,William K.L. Dickson,NaN,Thomas A. Edison,NaN,NaN,NaN


In [4]:
job_category.columns = ['tconst','actor', 'actress', 'archive_footage', 'archive_sound',
       'cinematographer', 'composer', 'director', 'editor', 'producer',
       'production_designer', 'self', 'writer']

In [5]:
movie = pd.read_csv('data/title_basic/data.tsv', sep ='	')
movie = movie[movie['titleType'] =='movie']

movie = movie.replace('\\N', np.nan)

movie['startYear'] = movie['startYear'].fillna(0).astype('int')
movie = movie[movie.startYear >= 1970].dropna(subset=['runtimeMinutes'])

movie
cross = pd.read_csv('data/title_principals/data.tsv', sep ='	')
cross = cross[['tconst', 'nconst']]

rate = pd.read_csv('data/title_rating/data.tsv', sep ='	')

movie = pd.merge(rate, movie, on='tconst')
start_yr = pd.read_csv('data/title_basic/data.tsv', sep ='	')
start_yr = start_yr[['tconst','startYear', 'titleType']]
cate = pd.read_csv('data/title_principals/data.tsv', sep ='	')
cate = pd.merge(cate, start_yr, on='tconst')
cate = cate[['tconst','nconst','category', 'startYear', 'titleType']]
cate = cate.replace('\\N', np.nan)


movie

C:\Users\kenny\.jupyter\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0015724,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,102,"Drama,Mystery,Romance"
1,tt0016906,5.6,15,movie,Frivolinas,Frivolinas,0,2014,NaN,80,"Comedy,Musical"
2,tt0031458,7.1,8,movie,El huésped del sevillano,El huésped del sevillano,0,1970,NaN,86,Comedy
3,tt0035423,6.4,76787,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
4,tt0036606,6.5,247,movie,"Another Time, Another Place","Another Time, Another Place",0,1983,NaN,118,"Drama,War"
5,tt0038687,7.4,1450,movie,Let There Be Light,Let There Be Light,0,1980,NaN,58,"Documentary,War"
6,tt0039442,6.2,125,movie,"Habla, mudita","Habla, mudita",0,1973,NaN,88,Drama
7,tt0054724,3.5,1191,movie,I Eat Your Skin,Zombie,0,1971,NaN,92,Horror
8,tt0057461,4.4,14,movie,La rosa de los vientos,La rosa de los vientos,0,1983,NaN,84,NaN
9,tt0058950,6.9,123,movie,"Auliban, the Seller of Rings",Biya el-Khawatim,0,1973,NaN,95,"Comedy,Musical"


In [6]:
cnts = pd.read_csv('data/movie_count_by_year.csv').groupby(['nconst','startYear','category'])['tconst'].max().unstack()
cnts

category             actor  actress  cinematographer  director  writer
nconst    startYear                                                   
nm0000001 1951         1.0      NaN              NaN       NaN     NaN
          1952         2.0      NaN              NaN       NaN     NaN
          1953         3.0      NaN              NaN       NaN     NaN
          1954         3.0      NaN              NaN       NaN     NaN
          1955         4.0      NaN              NaN       NaN     NaN
          1956         4.0      NaN              NaN       NaN     NaN
          1957         6.0      NaN              NaN       NaN     NaN
          1958         6.0      NaN              NaN       NaN     NaN
          1959         7.0      NaN              NaN       NaN     NaN
          1960         7.0      NaN              NaN       NaN     NaN
          1961         8.0      NaN              NaN       NaN     NaN
          1962         9.0      NaN              NaN       NaN     NaN
          1963         9.0      NaN              NaN       NaN     NaN
          1964         9.0      NaN              NaN       NaN     NaN
          1965         9.0      NaN              NaN       NaN     NaN
          1966         9.0      NaN              NaN       NaN     NaN
          1967         9.0      NaN              NaN       NaN     NaN
          1968        10.0      NaN              NaN       NaN     NaN
          1969        11.0      NaN              NaN       NaN     NaN
          1970        11.0      NaN              NaN       NaN     NaN
          1971        11.0      NaN              NaN       NaN     NaN
          1972        11.0      NaN              NaN       NaN     NaN
          1973        11.0      NaN              NaN       NaN     NaN
          1974        11.0      NaN              NaN       NaN     NaN
          1975        11.0      NaN              NaN       NaN     NaN
          1976        12.0      NaN              NaN       NaN     NaN
          1977        13.0      NaN              NaN       NaN     NaN
          1978        13.0      NaN              NaN       NaN     NaN
          1979        13.0      NaN              NaN       NaN     NaN
          1980        13.0      NaN              NaN       NaN     NaN
...                    ...      ...              ...       ...     ...
nm9993407 2010         NaN      NaN              NaN       1.0     NaN
          2011         NaN      NaN              NaN       1.0     NaN
          2012         NaN      NaN              NaN       1.0     NaN
          2013         NaN      NaN              NaN       1.0     NaN
          2014         NaN      NaN              NaN       1.0     NaN
          2015         NaN      NaN              NaN       1.0     NaN
          2016         NaN      NaN              NaN       1.0     NaN
          2017         NaN      NaN              NaN       1.0     NaN
          2018         NaN      NaN              NaN       1.0     NaN
          2019         NaN      NaN              NaN       1.0     NaN
          2020         NaN      NaN              NaN       1.0     NaN
nm9993444 2020         1.0      NaN              NaN       NaN     NaN
nm9993484 2019         NaN      1.0              NaN       NaN     NaN
          2020         NaN      1.0              NaN       NaN     NaN
nm9993485 2019         1.0      NaN              NaN       NaN     NaN
          2020         1.0      NaN              NaN       NaN     NaN
nm9993494 2018         NaN      NaN              NaN       NaN     1.0
          2019         NaN      NaN              NaN       NaN     1.0
          2020         NaN      NaN              NaN       NaN     1.0
nm9993616 2018         1.0      NaN              NaN       NaN     NaN
          2019         1.0      NaN              NaN       NaN     NaN
          2020         1.0      NaN              NaN       NaN     NaN
nm9993636 2020         1.0      NaN              NaN       NaN     NaN
nm9993650 2018         1

In [7]:
cnts.columns
movie = pd.merge(cross, movie, on='tconst')
del rate
del cross
del job_category
del exp
del start_yr

In [8]:
movie.head()

,tconst,nconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0015724,nm1457911,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,102,"Drama,Mystery,Romance"
1,tt0015724,nm0844752,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,102,"Drama,Mystery,Romance"
2,tt0015724,nm0869732,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,102,"Drama,Mystery,Romance"
3,tt0015724,nm0194720,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,102,"Drama,Mystery,Romance"
4,tt0015724,nm0650495,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,102,"Drama,Mystery,Romance"


In [9]:
movie = pd.merge(movie, cnts, left_on=['nconst', 'startYear'], right_index=True)
cate['startYear'] = cate['startYear'].fillna(0).astype('int')

cate = cate[(cate['category'].isin(['director','cinematographer','actor','actress','writer'])) & (cate['startYear'] > 1950)&(cate['startYear'] <= 2020)
           & (cate['titleType'] =='movie')].drop(columns='titleType')

collab = cate.groupby(['tconst', 'startYear', 'category'])['nconst'].max().unstack().groupby('director')['writer','cinematographer',
                                                                                               'actor','actress'].count()
collab.columns = ['writer_collab','cine_collab','actor_collab','actress_collab']
movie = pd.merge(movie, collab, how='left', left_on='nconst', right_index=True)


In [10]:
m = pd.read_csv('data/title_basic/data.tsv', sep ='	')
director = pd.read_csv('data/title_principals/data.tsv', sep ='	')
director = director[director['category'] == 'director']
director = pd.merge(director, m, on = 'tconst')
director = director.groupby(['nconst','genres'])['tconst'].count()

movie = pd.merge(movie, director, left_on=['nconst','genres'], right_index=True, how='left')

movie.head()

,tconst_x,nconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,...,actor,actress,cinematographer,director,writer,writer_collab,cine_collab,actor_collab,actress_collab,tconst_y
0,tt0015724,nm1457911,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,NaN,NaN,NaN,2.0,NaN,0.0,1.0,2.0,1.0,NaN
1,tt0015724,nm0844752,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0015724,nm0869732,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0015724,nm0194720,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,179.0
4,tt0015724,nm0650495,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
m = pd.read_csv('data/title_basic/data.tsv', sep ='	')
m = m[m['titleType'] =='movie']

m = m.replace('\\N', np.nan)

m['startYear'] = m['startYear'].fillna(0).astype('int')
m = m[m.startYear >= 1970].dropna(subset=['runtimeMinutes'])

m
cross = pd.read_csv('data/title_principals/data.tsv', sep ='	')
cross = cross[['tconst', 'nconst']]

rate2 = pd.read_csv('data/title_rating/data.tsv', sep ='	')

In [12]:
m = pd.merge(rate2, m, on='tconst')
m = pd.merge(cross, m, on='tconst')
res = []
for i in list(set(m.startYear)):
    weight = m[m.startYear <= i]
    weight['multiply'] = weight['averageRating']*weight['numVotes']
    mult = weight.groupby('nconst')['multiply'].sum()
    sum_votes = weight.groupby('nconst')['numVotes'].sum()
    weight = pd.merge(mult, sum_votes, right_index=True, left_index=True)
    weight['weight_avg'] = weight['multiply']/weight['numVotes']
    weight['startYear'] = i
    res.append(weight)
    

del m
del cross
del rate2

C:\Users\kenny\.jupyter\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
res = pd.concat(res).reset_index()

In [14]:
pd.merge(movie, res, on = ['nconst','startYear'])

,tconst_x,nconst,averageRating,numVotes_x,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,...,director,writer,writer_collab,cine_collab,actor_collab,actress_collab,tconst_y,multiply,numVotes_y,weight_avg
0,tt0015724,nm1457911,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,2.0,NaN,0.0,1.0,2.0,1.0,NaN,124.0,20,6.200000
1,tt0015724,nm0844752,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1054.3,189,5.578307
2,tt0015724,nm0869732,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,479.2,68,7.047059
3,tt0015724,nm0194720,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,179.0,124.0,20,6.200000
4,tt0015724,nm0650495,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124.0,20,6.200000
5,tt0015724,nm0529960,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,1.0,NaN,3.0,2.0,3.0,3.0,1.0,124.0,20,6.200000
6,tt0015724,nm1597742,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,124.0,20,6.200000
7,tt0015724,nm0600039,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,1.0,NaN,0.0,2.0,2.0,2.0,NaN,878.4,132,6.654545
8,tt0104751,nm0600039,6.8,94,movie,Lolo,Lolo,0,1993,NaN,...,1.0,NaN,0.0,2.0,2.0,2.0,NaN,878.4,132,6.654545
9,tt0015724,nm0006509,6.2,20,movie,Dama de noche,Dama de noche,0,1993,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223.2,36,6.200000


In [15]:

for i in movie.columns[12:]:
    tt = movie.groupby('tconst_x')[i].agg(['min', 'max', 'mean', 'sum', 'std'])
    tt = tt.rename(columns={'min' : str(i) + 'min_round', 'max' : str(i) + 'max_round', 'mean': str(i) + 'mean_round', 'sum': str(i) + 'sum_round', 'std': str(i) + 'std_round'})
    movie = pd.merge(movie, tt, left_on = 'tconst_x', right_index=True)

    movie[str(i) + 'min_round_diff'] = movie[i] - movie[str(i) + 'min_round']
    movie[str(i) + 'max_round_diff'] = movie[i] - movie[str(i) + 'max_round']
    movie[str(i) + 'std_round_diff'] = movie[i] - movie[str(i) + 'std_round']
    movie[str(i) + 'mean_round_diff'] = movie[i] - movie[str(i) + 'mean_round']

In [16]:
movie.columns[7:]

Index(['isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres', 'actor',
       'actress', 'cinematographer', 'director', 'writer',
       ...
       'actress_collabmean_round_diff', 'tconst_ymin_round',
       'tconst_ymax_round', 'tconst_ymean_round', 'tconst_ysum_round',
       'tconst_ystd_round', 'tconst_ymin_round_diff', 'tconst_ymax_round_diff',
       'tconst_ystd_round_diff', 'tconst_ymean_round_diff'],
      dtype='object', length=105)

In [17]:
movie = movie.drop(columns='endYear')
grp = movie.groupby(['tconst_x', 'averageRating', 'numVotes', 'runtimeMinutes','startYear','genres'])[movie.columns[7:]].max().reset_index()
#cnts = pd.concat([cnts, pd.get_dummies(cnts.genres, prefix_sep=',')], axis=1).drop(columns=['numVotes','genres', 'tconst','startYear']).fillna(0)
grp

,tconst_x,averageRating,numVotes,runtimeMinutes,startYear,genres,actor,actor_collab,actor_collabmax_round,actor_collabmax_round_diff,...,writer_collabsum_round,writermax_round,writermax_round_diff,writermean_round,writermean_round_diff,writermin_round,writermin_round_diff,writerstd_round,writerstd_round_diff,writersum_round
0,tt0015724,6.2,20,102,1993,"Drama,Mystery,Romance",10.0,3.0,3.0,0.0,...,3.0,1.0,0.0,1.000000,0.000000,1.0,0.0,NaN,NaN,1.0
1,tt0016906,5.6,15,80,2014,"Comedy,Musical",9.0,1.0,1.0,0.0,...,0.0,2.0,0.0,2.000000,0.000000,2.0,0.0,NaN,NaN,2.0
2,tt0031458,7.1,8,86,1970,Comedy,2.0,23.0,23.0,0.0,...,22.0,34.0,0.0,13.666667,20.333333,1.0,33.0,17.785762,16.214238,41.0
3,tt0035423,6.4,76787,118,2001,"Comedy,Fantasy,Romance",7.0,10.0,10.0,0.0,...,9.0,3.0,0.0,2.000000,1.000000,1.0,2.0,1.414214,1.585786,4.0
4,tt0036606,6.5,247,118,1983,"Drama,War",1.0,12.0,12.0,0.0,...,12.0,1.0,0.0,1.000000,0.000000,1.0,0.0,0.000000,1.000000,2.0
5,tt0038687,7.4,1450,58,1980,"Documentary,War",14.0,28.0,28.0,0.0,...,28.0,7.0,0.0,7.000000,0.000000,7.0,0.0,NaN,NaN,7.0
6,tt0039442,6.2,125,88,1973,Drama,82.0,22.0,22.0,0.0,...,38.0,3.0,0.0,2.000000,1.000000,1.0,2.0,1.414214,1.585786,4.0
7,tt0054724,3.5,1191,92,1971,Horror,3.0,4.0,4.0,0.0,...,2.0,1.0,0.0,1.000000,0.000000,1.0,0.0,NaN,NaN,1.0
8,tt0058950,6.9,123,95,1973,"Comedy,Musical",3.0,32.0,32.0,0.0,...,24.0,3.0,0.0,3.000000,0.000000,3.0,0.0,0.000000,3.000000,6.0
9,tt0058994,5.1,133,91,1976,"Action,Crime,Drama",48.0,22.0,22.0,0.0,...,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [18]:
grp.iloc[:, 8:] = grp.fillna(0).iloc[:, 8:].astype('int')
grp['runtimeMinutes'] = grp['runtimeMinutes'].astype('float64')

grp['averageRating'] = round(grp['averageRating'])

In [19]:
grp = grp.drop(columns=['numVotes','genres', 'tconst_x','startYear'])
grp.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171167 entries, 0 to 171166
Columns: 103 entries, averageRating to writersum_round
dtypes: float64(4), int32(99)
memory usage: 69.9 MB


In [37]:
X = grp.drop(columns='averageRating')
y = grp['averageRating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

xg_class = xgb.XGBRegressor()
xg_class.fit(X_train, y_train, verbose=5)

y_pred = xg_class.predict(X_test)


C:\Users\kenny\.jupyter\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[14:03:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [38]:
importance = pd.DataFrame({'Column Name': X_test.columns, 'Importance': xg_class.feature_importances_}).sort_values('Importance', ascending=False)
importance[importance['Column Name'] == 'director']

,Column Name,Importance
61,director,0.009514


### Accuracy Test

In [39]:
g = pd.concat([pd.DataFrame(y_pred).reset_index(drop=True), pd.DataFrame(y_test).reset_index(drop=True)],axis=1 )
g['bool'] = g.drop(columns=0).isin(g.averageRating).any(1)


In [40]:
def accuracy(x):
    if x[0] == x['averageRating']:
        return True
    else:
        return False
g['bool'] = g.apply(lambda d: accuracy(d), axis=1)
g.to_csv('data/review.csv')

In [41]:
g['bool'].value_counts()

False    42792
Name: bool, dtype: int64

In [42]:
g

,0,averageRating,bool
0,5.643451,3.0,False
1,5.522696,8.0,False
2,5.443987,5.0,False
3,5.524632,7.0,False
4,6.374660,8.0,False
5,5.901291,8.0,False
6,6.176727,6.0,False
7,5.655293,7.0,False
8,5.850034,8.0,False
9,6.013212,4.0,False


### Predicting unreleased movies

In [26]:
names = pd.read_csv('data/title_basic/data.tsv', sep ='	')
names = names[names['titleType']=='movie']
rate = pd.read_csv('data/title_rating/data.tsv', sep ='	')
names = pd.merge(names, rate, how='left', on='tconst')
names = names[pd.isnull(names.averageRating) == True].replace('\\N', np.nan)
names = names[(names.startYear == '2020') | (names.startYear == '2021')]

C:\Users\kenny\.jupyter\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
cross = pd.read_csv('data/title_principals/data.tsv', sep ='	')
cross = cross[['tconst', 'nconst']]
names = pd.merge(names, cross, on = 'tconst')
#n = pd.merge(n, cnts, left_on=['nconst', 'startYear'], right_index=True)

In [28]:
names

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,nconst
0,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm9731065
1,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm0269077
2,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm0655945
3,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm3580842
4,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm0300417
5,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm0004000
6,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm1860699
7,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm1291775
8,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm1164842
9,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm5220214


In [29]:
cnts_pred = pd.read_csv('data/movie_count_by_year.csv').groupby(['nconst','startYear','category'])['tconst'].max().unstack()
names['startYear'] = names['startYear'].astype('int64')
names = pd.merge(names, cnts_pred, left_on=['nconst', 'startYear'], right_index=True)
names

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,nconst,actor,actress,cinematographer,director,writer
0,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm9731065,NaN,NaN,1.0,NaN,NaN
1,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm0269077,14.0,NaN,NaN,NaN,NaN
2,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm0655945,NaN,5.0,NaN,NaN,NaN
3,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm3580842,3.0,NaN,NaN,NaN,NaN
4,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm0300417,8.0,NaN,NaN,NaN,NaN
5,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm0004000,NaN,NaN,NaN,1.0,2.0
6,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm1860699,NaN,NaN,NaN,NaN,1.0
7,tt0120589,movie,Arigo,Arigo,0,2020,NaN,NaN,Drama,NaN,NaN,nm1291775,NaN,NaN,NaN,NaN,3.0
58078,tt9422624,movie,Silvio Santos: O Sequestro,Silvio Santos: O Sequestro,0,2020,NaN,NaN,"Crime,Thriller",NaN,NaN,nm1291775,NaN,NaN,NaN,NaN,3.0
20,tt0368133,movie,The Promise of Perfume,The Promise of Perfume,0,2020,NaN,57,Documentary,NaN,NaN,nm9018863,NaN,1.0,NaN,NaN,NaN


In [30]:
names = pd.merge(names, collab, how='left', left_on='nconst', right_index=True)

In [31]:
# m = pd.read_csv('data/title_basic/data.tsv', sep ='	')
# director = pd.read_csv('data/title_principals/data.tsv', sep ='	')
# director = director[director['category'] == 'director']
# director = pd.merge(director, m, on = 'tconst')
# director = director.groupby(['nconst','genres'])['tconst'].count()

names = pd.merge(names, director, left_on=['nconst','genres'], right_index=True, how='left')
names = names[movie.columns[:21]]
names.head()

,tconst_x,nconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,...,actor,actress,cinematographer,director,writer,writer_collab,cine_collab,actor_collab,actress_collab,tconst_y
0,tt0120589,nm9731065,NaN,NaN,movie,Arigo,Arigo,0,2020,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0120589,nm0269077,NaN,NaN,movie,Arigo,Arigo,0,2020,NaN,...,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0120589,nm0655945,NaN,NaN,movie,Arigo,Arigo,0,2020,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0120589,nm3580842,NaN,NaN,movie,Arigo,Arigo,0,2020,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0120589,nm0300417,NaN,NaN,movie,Arigo,Arigo,0,2020,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
names.head()

In [ ]:
names.columns[11:]

In [ ]:
for i in names.columns[11:]:
    tt = names.groupby('tconst_x')[i].agg(['min', 'max', 'mean', 'sum', 'std'])
    tt = tt.rename(columns={'min' : str(i) + 'min_round', 'max' : str(i) + 'max_round', 'mean': str(i) + 'mean_round', 'sum': str(i) + 'sum_round', 'std': str(i) + 'std_round'})
    names = pd.merge(names, tt, left_on = 'tconst_x', right_index=True)

    names[str(i) + 'min_round_diff'] = names[i] - names[str(i) + 'min_round']
    names[str(i) + 'max_round_diff'] = names[i] - names[str(i) + 'max_round']
    names[str(i) + 'std_round_diff'] = names[i] - names[str(i) + 'std_round']
    names[str(i) + 'mean_round_diff'] = names[i] - names[str(i) + 'mean_round']

In [ ]:

names = names.fillna(0).groupby(['tconst_x','primaryTitle', 'numVotes', 'runtimeMinutes','startYear','genres'])[names.columns[7:]].max().reset_index()

names.iloc[:, 8:] = names.fillna(0).iloc[:, 8:].astype('int')
names['runtimeMinutes'] = names['runtimeMinutes'].astype('float64')
upcoming_movies = names.drop(columns=['numVotes','genres', 'tconst_x','startYear','primaryTitle'])


In [ ]:
upcoming_movies = upcoming_movies[X.columns]
y_pred = xg_class.predict(upcoming_movies)


In [ ]:
final_preds = pd.concat([names, pd.DataFrame(y_pred)], axis=1)


final_preds[final_preds['primaryTitle'] == 'Tenet']

## Scratch

In [ ]:
start_yr = pd.read_csv('data/title_basic/data.tsv', sep ='	')
start_yr = start_yr[['tconst','startYear', 'titleType']]
cate = pd.read_csv('data/title_principals/data.tsv', sep ='	')
cate = pd.merge(cate, start_yr, on='tconst')
cate = cate[['tconst','nconst','category', 'startYear', 'titleType']]
cate = cate.replace('\\N', np.nan)

cate['startYear'] = cate['startYear'].fillna(0).astype('int')

In [ ]:
cate = cate[(cate['category'].isin(['director','cinematographer','actor','actress','writer'])) & (cate['startYear'] > 1950)&(cate['startYear'] <= 2020)
           & (cate['titleType'] =='movie')].drop(columns='titleType')

In [ ]:


cate = cate.replace('\\N', np.nan)

cate['startYear'] = cate['startYear'].fillna(0).astype('int')
cate = cate[cate.startYear > 0]
movie_counts_by_year = []
for i in list(set(cate.startYear)):
    filtered_year = cate[cate.startYear <= i]
    grp_cnt = pd.DataFrame(filtered_year.groupby(['nconst', 'category'])['tconst'].count())
    grp_cnt['startYear'] = i
    #grp_cnt.to_csv('data/mov_cnt/movie_cnt_'+str(i)+'.csv')
    movie_counts_by_year.append(grp_cnt)
    
movie_counts_by_year = pd.concat(movie_counts_by_year)

In [ ]:
#movie_counts_by_year.to_csv('data/movie_count_by_year.csv')

In [ ]:
movie_counts_by_year.head()

In [ ]:
cate = cate.replace('\\N', np.nan)

cate['startYear'] = cate['startYear'].fillna(0).astype('int')
cate = cate[cate.startYear > 0]
movie_counts_by_year = []
for i in list(set(cate.startYear)):
    filtered_year = cate[cate.startYear <= i]
    grp_cnt = pd.DataFrame(filtered_year.groupby(['nconst', 'category'])['tconst'].count())
    grp_cnt['startYear'] = i
    #grp_cnt.to_csv('data/mov_cnt/movie_cnt_'+str(i)+'.csv')
    movie_counts_by_year.append(grp_cnt)
    
movie_counts_by_year = pd.concat(movie_counts_by_year)

In [ ]:
m = pd.read_csv('data/title_basic/data.tsv', sep ='	')
m = m[m['titleType'] =='movie']

m = m.replace('\\N', np.nan)

m['startYear'] = m['startYear'].fillna(0).astype('int')
m = m[m.startYear >= 1970].dropna(subset=['runtimeMinutes'])

m
cross = pd.read_csv('data/title_principals/data.tsv', sep ='	')
cross = cross[['tconst', 'nconst']]

rate2 = pd.read_csv('data/title_rating/data.tsv', sep ='	')

m = pd.merge(rate2, m, on='tconst')
m = pd.merge(cross, m, on='tconst')

In [ ]:

m['multiply'] = m['averageRating']*m['numVotes']
mult = m.groupby('nconst')['multiply'].sum()
sum_votes = m.groupby('nconst')['numVotes'].sum()
m = pd.merge(mult, sum_votes, right_index=True, left_index=True)
m['weight_avg'] = m['multiply']/m['numVotes']
del
m.head()

In [ ]:
8*500

In [ ]:
700+4000

In [ ]:
4700/600